In [ ]:
# === Environment Setup ===
import os, sys, math, time, random, json, textwrap, warnings
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
# pip install geopandas mapclassify libpysal esda splot
try:
    import geopandas as gpd
    from shapely.geometry import Point
    import mapclassify
    import libpysal as lps
    from esda.moran import Moran
    from splot.esda import moran_scatterplot
    PYSAL_AVAILABLE = True
except ImportError:
    PYSAL_AVAILABLE = False
from IPython.display import display, Markdown

# --- Configuration ---
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({'font.size': 12, 'figure.figsize': (11, 7), 'figure.dpi': 130})
np.set_printoptions(suppress=True, linewidth=120, precision=4)

# --- Utility Functions ---
def note(msg): display(Markdown(f"<div class='alert alert-block alert-info'>📝 **Note:** {msg}</div>"))
def sec(title): print(f"\n{80*'='}\n| {title.upper()} |\n{80*'='}")

if not PYSAL_AVAILABLE: 
    import subprocess
    subprocess.run(['pip', 'install', 'geopandas', 'mapclassify', 'libpysal', 'esda', 'splot'])
    note("Key geospatial libraries (`geopandas`, `pysal`) not found. Some sections will be skipped. Run `pip install geopandas mapclassify libpysal esda splot`")
note(f"Environment initialized. GeoPandas and PySAL available: {PYSAL_AVAILABLE}")

# Part 7: Advanced Topics
## Chapter 7.20: Geospatial Data Analysis

### Introduction: The Spatial Dimension of Economics

Economic activity is not placeless; it happens *somewhere*. The analysis of geographic data is a cornerstone of urban economics, development economics, international trade, environmental economics, and economic history. The rise of new data sources—high-resolution satellite imagery, detailed administrative boundaries from government agencies, and GPS data from mobile devices—has led to a torrent of new data with a spatial component. Understanding how to work with this data is a critical skill for the modern economist.

This notebook introduces the fundamentals of geospatial analysis using Python's premier tool: **`GeoPandas`**. `GeoPandas` extends the familiar `pandas` DataFrame to allow for spatial operations, making it intuitive for anyone comfortable with standard data analysis. We will cover:

1.  **Core Concepts:** Understanding vector data, the critical importance of Coordinate Reference Systems (CRS), and the `shapely` geometry objects that power `GeoPandas`.
2.  **Choropleth Mapping:** Creating thematic maps to visualize spatial patterns and the importance of data classification.
3.  **Spatial Joins:** The workhorse operation for merging datasets based on their spatial relationships.
4.  **Spatial Autocorrelation:** Moving beyond visualization to formally test for spatial dependence and clustering using statistics like Moran's I.

## 1. Core Concepts of Geospatial Data

### 1.1 Vector Data and `shapely` Geometries
The most common type of geospatial data used in economics is **vector data**, which represents the world using discrete geometric objects defined by vertices. These are the fundamental building blocks:
- **Points:** Represent a single location, like a firm, a household, a city, or a specific pollution monitor.
- **Lines:** Represent linear features, like a river, road, or pipeline.
- **Polygons:** Represent areas, like a country's boundary, a census tract, a property parcel, or a zoning district.

`GeoPandas` stores these geometries in a special `geometry` column. Each entry in this column is a `shapely` object, which endows the DataFrame with spatial awareness and enables geometric operations.

### 1.2 Coordinate Reference Systems (CRS)
A **Coordinate Reference System (CRS)** is the metadata that connects the 2D coordinates of your map data to a specific location on the 3D Earth. This is arguably the single most important concept in applied geospatial analysis. Any projection from a 3D sphere onto a 2D map inevitably introduces distortions. Different CRSs are designed to preserve different properties (e.g., area, shape, distance) for specific regions of the globe. It is absolutely critical that **all datasets are in the same CRS before they can be combined or analyzed together.**

- **Geographic CRS:** Uses latitude and longitude on a 3D ellipsoid model of the Earth. The most common is **WGS 84 (EPSG:4326)**, the standard for GPS. While excellent for locating points, it severely distorts area and distance, especially near the poles. For example, on a standard WGS 84 map, Greenland looks larger than Africa, when in reality Africa is 14 times larger.
- **Projected CRS:** A 2D projection of the 3D Earth. These are essential for analysis because they can preserve key properties for a specific region. For example, an **equal-area projection** (like Mollweide or Albers Equal Area) is required if you want to accurately calculate population density or land use shares.

In [ ]:
sec("Visualizing CRS Distortions")

if PYSAL_AVAILABLE:
    # Load the world dataset, which is in WGS 84 (lat/lon) by default
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    # Remove Antarctica for a clearer visualization of the distortion effect
    world = world[(world.name != "Antarctica")]

    fig, axes = plt.subplots(1, 2, figsize=(18, 7), subplot_kw={'aspect':'equal'})
    fig.suptitle('Figure 1: The Impact of Coordinate Reference System Choice', fontsize=20, y=1.0)

    # Plot 1: Standard WGS 84 (EPSG:4326)
    # Notice how Greenland looks enormous, comparable in size to Africa.
    world.plot(ax=axes[0])
    axes[0].set_title('a) Geographic CRS (WGS 84): Distorts Area', fontsize=14)
    axes[0].set_xlabel("Longitude"); axes[0].set_ylabel("Latitude")

    # Plot 2: Mollweide Equal-Area Projection
    # Re-projecting to an equal-area CRS shows the true relative sizes.
    # The to_crs() method is the standard way to change projections.
    world.to_crs('+proj=moll').plot(ax=axes[1])
    axes[1].set_title('b) Projected CRS (Mollweide): Preserves Area', fontsize=14)
    axes[1].set_xticklabels([]); axes[1].set_yticklabels([])
    axes[1].set_xlabel("X Coordinate (meters)"); axes[1].set_ylabel("Y Coordinate (meters)")

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

    note("The choice of CRS is not just a cosmetic issue. Using WGS 84 to calculate the area of Greenland would lead to a massive overestimation. For economic analysis involving area or distance, always project your data to a suitable CRS first.")
else:
    note("Geospatial libraries not installed. Skipping CRS visualization.")

## 2. Choropleth Maps with Data Classification

A **choropleth map** is a thematic map where areas are shaded in proportion to a statistical variable (e.g., GDP per capita, unemployment rate). To create meaningful and honest visualizations, it is crucial to classify continuous data into discrete bins. A continuous color scale can be misleading, especially for skewed data, as extreme outliers can "drown out" all the variation in the rest of the data, rendering most of the map a single color.

The `mapclassify` library, which integrates with `geopandas`, provides several standard classification schemes:
- **Quantiles:** Each bin contains the same number of observations (e.g., quintiles, deciles). This is good for seeing relative rankings and is robust to outliers.
- **Equal Interval:** Each bin has the same width in data space (e.g., 0-1000, 1000-2000, ...). This is simple to interpret but can be heavily skewed by outliers.
- **Fisher-Jenks:** A computational method that identifies "natural breaks" in the data by finding class boundaries that minimize the variance within each class and maximize the variance between classes. It is often effective at revealing underlying data structure.

In [ ]:
sec("Creating a Choropleth Map")

if PYSAL_AVAILABLE:
    # Calculate GDP per capita for the visualization
    world['gdp_per_cap'] = world['gdp_md_est'] / world['pop_est']
    world_filtered = world[(world.pop_est > 0) & (world.name != 'Antarctica')].copy()

    fig, ax = plt.subplots(1, 1, figsize=(16, 10))

    world_filtered.plot(
        column='gdp_per_cap',   # The data to be visualized
        ax=ax, 
        legend=True,
        cmap='viridis_r',       # Use a reversed colormap so high values are dark
        scheme='Quantiles',     # Use the Quantiles classification scheme
        k=5,                    # Classify the data into 5 bins (quintiles)
        legend_kwds={'title': "GDP per Capita (USD, Quantiles)",
                     'bbox_to_anchor': (0.5, -0.05), # Position legend below the map
                     'loc': 'lower center', 'ncol': 5}
    )

    ax.set_title("Figure 2: World GDP per Capita", fontsize=18)
    ax.set_axis_off() # Remove the lat/lon axes for a cleaner map
    plt.show()
else:
    note("Geospatial libraries not installed. Skipping Choropleth map.")

## 3. Spatial Joins and Spatial Autocorrelation

### 3.1 Spatial Joins
A **spatial join** is one of the most powerful tools in geospatial analysis. It merges two `GeoDataFrames` based on their spatial relationship (e.g., intersects, contains, is within), rather than on a common column like a standard `pandas` merge. This is the key operation for combining different sources of data. For example, we can join a `GeoDataFrame` of firm locations (Points) with a `GeoDataFrame` of census tracts (Polygons) to attach local demographic information (income, population) to each firm.

In [ ]:
sec("Spatial Join: Attributing Cities to Countries")

if PYSAL_AVAILABLE:
    # Load a dataset of world cities (Points)
    cities = gpd.read_file(gpd.datasets.get_path('naturalearth_cities'))
    note(f"Loaded {len(cities)} cities.")

    # It is CRITICAL that both GeoDataFrames use the same CRS before joining
    assert world.crs == cities.crs, "CRS mismatch between datasets!"

    # Perform the spatial join. For each city (point), find the country (polygon) it is 'within'.
    # The resulting GeoDataFrame has the geometry of the first (left) GDF and the attributes of both.
    cities_with_country_info = gpd.sjoin(cities, world, how="inner", predicate='within')

    note(f"{len(cities_with_country_info)} cities were successfully joined to a country.")
    print("\n--- Sample of Cities after Spatial Join (with country attributes) ---")
    # Note that the columns from the 'world' GeoDataFrame (like gdp_per_cap) are now available for the cities.
    display(cities_with_country_info[['name', 'gdp_per_cap', 'continent']].head())
else:
    note("Geospatial libraries not installed. Skipping Spatial Join example.")

### 3.2 Spatial Autocorrelation
**Tobler's First Law of Geography** states: "Everything is related to everything else, but near things are more related than distant things." **Spatial autocorrelation** is the formal statistical measure of this phenomenon. It tests whether the value of a variable at a given location is correlated with the values of that variable in neighboring locations.

To measure it, we first must formally define a **spatial weights matrix (W)**, which is an $N \times N$ matrix that specifies the "neighbor" relationships for each location. Common definitions include:
- **Contiguity:** Two polygons are neighbors if they share a border (Rook contiguity) or a border/vertex (Queen contiguity).
- **Distance-based:** Two points are neighbors if they are within a certain distance threshold.

With **W**, we can calculate statistics like **Moran's I**, which indicates the type and strength of spatial autocorrelation.
- **Moran's I > 0:** Indicates positive spatial autocorrelation (spatial clustering of similar values, e.g., rich countries are near other rich countries).
- **Moran's I < 0:** Indicates negative spatial autocorrelation (a checkerboard pattern of dissimilar values).
- **Moran's I ≈ 0:** Indicates no spatial pattern (spatial randomness).

In [ ]:
sec("Testing for Spatial Autocorrelation of GDP")

if PYSAL_AVAILABLE:
    # Prepare data, dropping NAs. It is common to use log-transformed variables.
    data = world_filtered[['gdp_per_cap', 'geometry']].dropna().copy()
    data['log_gdp_pc'] = np.log(data.gdp_per_cap)
    
    # 1. Create a spatial weights matrix based on Queen contiguity.
    w = lps.weights.Queen.from_dataframe(data)
    # Row-standardize the weights matrix. This means the influence of each neighbor is proportional.
    w.transform = 'r'

    # 2. Calculate Moran's I statistic
    moran = Moran(data['log_gdp_pc'], w)
    
    # 3. Print and interpret the result
    note(f"Moran's I statistic: {moran.I:.4f} (p-value under simulation: {moran.p_sim:.4f})")
    note("The positive and highly significant Moran's I suggests strong spatial clustering. High-GDP countries tend to be located near other high-GDP countries, and low-GDP countries near other low-GDP ones.")
    
    # 4. Create a Moran Scatter Plot to visualize the spatial relationship
    fig, ax = plt.subplots(1, figsize=(8, 8))
    moran_scatterplot(moran, ax=ax)
    ax.set(xlabel='Log GDP per Capita (Standardized)', ylabel='Spatially Lagged Log GDP per Capita (Neighbors\' Average)')
    plt.suptitle("Figure 3: Moran Scatterplot for GDP per Capita", fontsize=16)
    plt.show()
    note("The plot shows a strong positive linear relationship. Most countries are in the High-High (top-right) or Low-Low (bottom-left) quadrants, confirming the visual pattern of clustering. The slope of the regression line is Moran's I.")
else:
    note("PySAL not found. Skipping spatial autocorrelation example. Run `pip install pysal`.")

## 4. Exercises

1.  **CRS Transformation & Area Calculation:** Load the `naturalearth_lowres` world dataset. Calculate the area of each country polygon using the `.area` attribute. Which country does it report as having the largest area? Now, re-project the `GeoDataFrame` to an equal-area projection like Mollweide (`world.to_crs('+proj=moll')`). Recalculate the area (which will now be in square meters). Does the country with the largest area change? Why is calculating area in a geographic CRS (like WGS 84) misleading for economic comparisons?

2.  **Choropleth Classification Schemes:** Create a choropleth map of world population (`pop_est`). Plot three versions side-by-side, using 'Quantiles', 'EqualInterval', and 'JenksCaspall' classification schemes (`scheme=...`). How does the visual story of population distribution change with each scheme? Which one do you think is most effective for this highly skewed data, and why?

3.  **Spatial Join Analysis:** Using the `cities_with_country_info` `GeoDataFrame` from section 3.1, calculate the average GDP per capita of the countries that contain the cities in our dataset. Is this average higher or lower than the global average GDP per capita across all countries in the `world` dataset? What might this suggest about the sample of cities included in the `naturalearth_cities` dataset (i.e., is it a representative sample of all cities)?

4.  **Moran's I for Population Density:** Repeat the spatial autocorrelation analysis from Section 3.2, but this time use the logarithm of population density (`pop_est` divided by the country's area) as your variable of interest. Remember to use an equal-area projection to calculate the area accurately! Calculate Moran's I and interpret the result. Are densely populated countries clustered together?

5.  **Real-World Data Application:** Download a shapefile of US counties from a public source (like the US Census Bureau's TIGER/Line files). Find a separate CSV file with some county-level economic data (e.g., unemployment rate, median income from the American Community Survey). Load both datasets, perform a standard `pandas` merge on the county FIPS code to combine them into a `GeoDataFrame`, and then create a choropleth map of your chosen economic variable. Finally, test for and interpret the spatial autocorrelation in that variable.